In [1]:
import fml_lib
import numpy as np
import pandas as pd
import scipy.stats as ss
import os.path
from math import sqrt, exp, log
import matplotlib.pyplot as plt
from mp_engine import mpPandasObj

In [2]:
dollar_bars = pd.read_csv("USA500IDXUSD_bars.csv", parse_dates=[['Date', 'Timestamp']])
dollar_bars = dollar_bars.set_index("Date_Timestamp")
close = dollar_bars['Close']
ffd_close = np.log(dollar_bars[['Close']])
daily_volatility = fml_lib.getDailyVol(dollar_bars.Close,span0=100)


C:\Users\dstek\AppData\Local\Temp\ipykernel_20520\2489997740.py:1: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  dollar_bars = pd.read_csv("USA500IDXUSD_bars.csv", parse_dates=[['Date', 'Timestamp']])


In [3]:
for name in ffd_close.columns:
        print(name)

Close


In [4]:
test_ffd = fml_lib.fracDiff_FFD(ffd_close,0.5)
print(test_ffd)

926


c:\Users\dstek\Documents\ECE479\fml_lib.py:276: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  seriesF,df_=series[[name]].fillna(method='ffill').dropna(),pd.Series()


KeyboardInterrupt: 

In [ ]:

if os.path.exists("./t_events.csv"):
    t_events = pd.read_csv("t_events.csv", index_col=[0])
    t_events = t_events.index
else:
    t_events = fml_lib.getTEvents(dollar_bars.Close, daily_volatility)
    t_events_df = pd.DataFrame(index= t_events)
    t_events_df.to_csv("./t_events_csv", index=True)

In [ ]:
t_events = pd.to_datetime(t_events)
print(t_events)

DatetimeIndex(['2023-07-04 22:00:10', '2023-07-04 22:00:40',
               '2023-07-04 22:00:50', '2023-07-04 22:01:10',
               '2023-07-04 22:01:30', '2023-07-04 22:01:50',
               '2023-07-04 22:02:10', '2023-07-04 22:02:20',
               '2023-07-04 22:02:40', '2023-07-04 22:03:00',
               ...
               '2024-01-25 03:52:20', '2024-01-25 03:53:40',
               '2024-01-25 03:54:50', '2024-01-25 03:55:10',
               '2024-01-25 03:55:40', '2024-01-25 03:56:50',
               '2024-01-25 03:57:00', '2024-01-25 03:57:20',
               '2024-01-25 03:58:30', '2024-01-25 03:58:40'],
              dtype='datetime64[ns]', length=814094, freq=None)


In [ ]:


#adds the vertical barrier num_days out from the time of the event
numDays = 10
t1=close.index.searchsorted(t_events+pd.Timedelta(days=numDays))
t1=t1[t1<close.shape[0]]
t1=pd.Series(close.index[t1],index=t_events[:t1.shape[0]]) # NaNs at end

In [ ]:
min_ret= 0.005
trgt = pd.Series(0.01, index=t_events)
if os.path.exists("./events.csv"):
    events = pd.read_csv("events.csv", index_col=[0])
    events = events.dropna()
else:
    events = fml_lib.getEvents(close, t_events, 0.02,trgt ,min_ret,t1)
    events.to_csv("events.csv")

In [ ]:

out = fml_lib.getBins(events, close, numDays)

[ True  True  True ...  True  True  True]


Want to find sample weights for use in the boosting fit

In [ ]:
temp = list(set(pd.to_datetime(events.t1.index)) & set(pd.to_datetime(events.index)) )

In [ ]:
#test_co_events = mpPandasObj(fml_lib.mpNumCoEvents,('molecule',events.index),24, closeIdx=close.index,t1=events['t1'])

if os.path.exists("./t_events.csv"):  
    numCoEvents = pd.read_csv("./co_events.csv", index_col=[0])
else:
    numCoEvents = fml_lib.get_co_events(close,events)
    numCoEvents.to_csv("./co_events.csv", index=True)

In [ ]:
ret=np.log(close).diff() # log-returns, so that they are additive

In [ ]:

old_index = numCoEvents.index
old_co = numCoEvents.values

In [ ]:
numCoEvents=numCoEvents.loc[~numCoEvents.index.duplicated(keep='last')]
numCoEvents=numCoEvents.reindex(close.index,fill_value=0)
numCoEvents.loc[old_index] = old_co
print(numCoEvents.loc[old_index])
print(numCoEvents)

                      0
Date_Timestamp         
2023-07-04 22:00:10   1
2023-07-04 22:00:40   2
2023-07-04 22:00:50   3
2023-07-04 22:01:10   4
2023-07-04 22:01:30   5
...                  ..
2024-01-25 03:53:40  42
2024-01-25 03:54:50  38
2024-01-25 03:55:10  17
2024-01-25 03:55:40  17
2024-01-25 03:56:50  17

[837059 rows x 1 columns]
                     0
Date_Timestamp        
2023-07-02 22:00:00  0
2023-07-02 22:00:10  0
2023-07-02 22:00:20  0
2023-07-02 22:00:30  0
2023-07-02 22:00:40  0
...                 ..
2024-01-25 03:57:00  0
2024-01-25 03:57:10  0
2024-01-25 03:57:20  0
2024-01-25 03:58:30  0
2024-01-25 03:58:40  0

[840953 rows x 1 columns]


In [ ]:
numThreads = 12

#out['tW']=mpPandasObj(fml_lib.mpSampleTW,('molecule',events.index),numThreads, t1=events['t1'],numCoEvents=numCoEvents)

In [ ]:

out['w'] = mpPandasObj(fml_lib.mpSampleW,('molecule',events.index),numThreads, t1=events['t1'],numCoEvents=numCoEvents,close=close)
print(out['w'])

2024-04-15 15:23:34.663612 100.0% mpSampleW done after 3.36 minutes. Remaining 0.0 minutes...
c:\Users\dstek\Documents\ECE479\mp_engine.py:123: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df0=df0._append(i)


2023-07-04 22:00:10    0.000883
2023-07-04 22:00:40    0.000719
2023-07-04 22:00:50    0.000427
2023-07-04 22:01:10    0.000652
2023-07-04 22:01:30    0.000475
                         ...   
2024-01-25 03:50:30    0.000077
2024-01-25 03:51:30    0.000029
2024-01-25 03:51:40    0.000083
2024-01-25 03:51:50    0.000020
2024-01-25 03:52:00    0.000070
Name: w, Length: 814081, dtype: float64


In [ ]:
test = fml_lib.getTimeDecay(out['w'])

0.30544824265189746


In [ ]:
def plotMinFFD():
    from statsmodels.tsa.stattools import adfuller
    import matplotlib.pyplot as mpl
    out=pd.DataFrame(columns=['adfStat','pVal','lags','nObs','95% conf','corr'])
    df0=dollar_bars
    for d in np.linspace(0,1,11):
        df1=np.log(df0[['Close']]).resample('1D').last() # downcast to daily obs
        print(df1)
        df2=fml_lib.fracDiff_FFD(df1,d,thres=.01)
        corr=np.corrcoef(df1.loc[df2.index,'Close'],df2['Close'])[0,1]
        df2=adfuller(df2['Close'],maxlag=1,regression='c',autolag=None)
        print(df2)
        out.loc[d]=list(df2[:4])+[df2[4]['5%']]+[corr] # with critical value
    out.to_csv('./_testMinFFD.csv')
    out[['adfStat','corr']].plot(secondary_y='adfStat')
    mpl.axhline(out['95% conf'].mean(),linewidth=1,color='r',linestyle='dotted')
    mpl.savefig('./_testMinFFD.png')
    return

In [ ]:
plotMinFFD()

                   Close
Date_Timestamp          
2023-07-02      8.400957
2023-07-03      8.401529
2023-07-04      8.401081
2023-07-05      8.399122
2023-07-06      8.391494
...                  ...
2024-01-21      8.485420
2024-01-22      8.486843
2024-01-23      8.491694
2024-01-24      8.490979
2024-01-25      8.490516

[208 rows x 1 columns]
(-0.35107168270351874, 0.9179531322919451, 1, 175, {'1%': -3.4682803641749267, '5%': -2.8782017240816327, '10%': -2.5756525795918366})
                   Close
Date_Timestamp          
2023-07-02      8.400957
2023-07-03      8.401529
2023-07-04      8.401081
2023-07-05      8.399122
2023-07-06      8.391494
...                  ...
2024-01-21      8.485420
2024-01-22      8.486843
2024-01-23      8.491694
2024-01-24      8.490979
2024-01-25      8.490516

[208 rows x 1 columns]


c:\Users\dstek\Documents\ECE479\fml_lib.py:275: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  seriesF,df_=series[[name]].fillna(method='ffill').dropna(),pd.Series()
c:\Users\dstek\Documents\ECE479\fml_lib.py:275: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  seriesF,df_=series[[name]].fillna(method='ffill').dropna(),pd.Series()
C:\Users\dstek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)


ZeroDivisionError: division by zero

In [ ]:
print(out)

                          ret  bin         w
2023-07-04 22:00:10  0.000226  1.0  0.000883
2023-07-04 22:00:40  0.000223  1.0  0.000719
2023-07-04 22:00:50  0.000220  1.0  0.000427
2023-07-04 22:01:10  0.000226  1.0  0.000652
2023-07-04 22:01:30  0.000224  1.0  0.000475
...                       ...  ...       ...
2024-01-25 03:50:30  0.000207  1.0  0.000077
2024-01-25 03:51:30  0.000216  1.0  0.000029
2024-01-25 03:51:40  0.000208  1.0  0.000083
2024-01-25 03:51:50  0.000215  1.0  0.000020
2024-01-25 03:52:00  0.000207  1.0  0.000070

[814081 rows x 3 columns]


In [ ]:
test_diff = fml_lib.fracDiff_FFD(out,0.5)
print(test_diff)

c:\Users\dstek\Documents\ECE479\fml_lib.py:275: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  seriesF,df_=series[[name]].fillna(method='ffill').dropna(),pd.Series()


KeyboardInterrupt: 